## Requirements
- openai （需要 1.x 版）
- Azure OpenAI 助理目前位於瑞典中部、美國東部 2 和澳大利亞東部。 如需這些區域中模型可用性的詳細資訊，請參閱 [模型指南](https://learn.microsoft.com/zh-tw/azure/ai-services/openai/concepts/models#assistants-preview)
- API Version 在 2024-02-15-preview 之後
- 模型：`gpt-35-turbo (1106)` 或 `gpt-4 (1106-preview)` 

In [2]:
from pydantic_settings import BaseSettings
from typing import Optional


class AOAISettings(BaseSettings):
    AOAI_API_KEY: str
    AOAI_API_VERSION: str
    AOAI_API_ENDPOINT: str
    AOAI_GPT3_MODEL: str
    AOAI_GPT4_MODEL: Optional[str] = ""
    AOAI_EMBEDDING_MODEL: str


class Env(AOAISettings):
    class Config:
        env_file = ".env"

env = Env()

In [3]:
from openai import AzureOpenAI

aoai = AzureOpenAI(
    api_key=env.AOAI_API_KEY,
    api_version='2024-02-15-preview',
    azure_endpoint=env.AOAI_API_ENDPOINT,
)

## Code Interceptor
- [Reference](https://learn.microsoft.com/zh-tw/azure/ai-services/openai/how-to/code-interpreter?tabs=python)
- 需要額外的 [Pricing](https://azure.microsoft.com/zh-tw/pricing/details/cognitive-services/openai-service/)

In [4]:
import time


if env.AOAI_GPT4_MODEL:
    # Create an assistant
    assistant = aoai.beta.assistants.create(
        name="Math Assist",
        instructions="You are an AI assistant that can write code to help answer math questions.",
        tools=[{"type": "code_interpreter"}],
        model=env.AOAI_GPT4_MODEL,  # You must replace this value with the deployment name for your model.
    )

    # Create a thread
    thread = aoai.beta.threads.create()

    # Add a user question to the thread
    message = aoai.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
    )

    # Run the thread
    start_time = time.perf_counter()
    run = aoai.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    # Retrieve the status of the run
    run = aoai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    status = run.status

    # Wait till the assistant has responded
    while status not in ["completed", "cancelled", "expired", "failed"]:
        time.sleep(5)
        run = aoai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        status = run.status

    messages = aoai.beta.threads.messages.list(thread_id=thread.id)

    print(f'execute_time: {time.perf_counter() - start_time}s')
    print(messages.model_dump_json(indent=2))

execute_time: 86.88521178500001
{
  "data": [
    {
      "id": "msg_08oyg7jqPBkZqZeQBdOpXFDz",
      "assistant_id": "asst_F8LB4l9Zz9ns1FUT3BIXljkx",
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "It seems there was an internal error when attempting to solve the equation using the code execution environment. However, I can still explain how to solve the equation `3x + 11 = 14`.\n\nTo solve the equation `3x + 11 = 14`, we can follow these steps:\n\n1. Subtract 11 from both sides of the equation to isolate the term with `x` on one side of the equation:\n\n   `3x + 11 - 11 = 14 - 11`\n   \n   Simplifying both sides of the equation, we get:\n   \n   `3x = 3`\n\n2. Divide both sides by 3 to solve for `x`:\n\n   `3x / 3 = 3 / 3`\n   \n   Simplifying, we find:\n   \n   `x = 1`\n\nSo, the solution is `x = 1`."
          },
          "type": "text"
        }
      ],
      "created_at"

## Function Calling

In [5]:
import time

get_current_weather = {
    "type": "function",
    "function": {
        "name": "getCurrentWeather",
        "description": "Get the weather in location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["c", "f"]},
            },
            "required": ["location"],
        },
    },
}

get_nickname = {
    "type": "function",
    "function": {
        "name": "getNickname",
        "description": "Get the nickname of a city",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state e.g. San Francisco, CA",
                },
            },
            "required": ["location"],
        },
    },
}

if env.AOAI_GPT4_MODEL:

    assistant = aoai.beta.assistants.create(
        instructions="You are a weather bot. Use the provided functions to answer questions.",
        model=env.AOAI_GPT4_MODEL,  # Replace with model deployment name
        tools=[get_current_weather, get_nickname],
    )

    # Create a thread
    thread = aoai.beta.threads.create()

    # Add a user question to the thread
    message = aoai.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content="台北天氣如何？",
    )

    start_time = time.perf_counter()
    # Run the thread
    run = aoai.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )

    # Retrieve the status of the run
    run = aoai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    status = run.status

    # Wait till the assistant has responded
    while status not in ["completed", "cancelled", "expired", "failed"]:
        time.sleep(5)
        if run.required_action:
            tool_call_id = run.required_action.submit_tool_outputs.tool_calls[0].id
            function_name = run.required_action.submit_tool_outputs.tool_calls[
                0
            ].function.name
            run = aoai.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=[
                    {
                        "tool_call_id": tool_call_id,
                        "output": "22C",
                    }
                ],
            )

        run = aoai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        status = run.status

    messages = aoai.beta.threads.messages.list(thread_id=thread.id)

    print(f'execute_time: {time.perf_counter() - start_time}s')
    print(messages.model_dump_json(indent=2))

execute_time: 22.482898265999992
{
  "data": [
    {
      "id": "msg_DKAUtV7GTy8uCOmJlKJINKy2",
      "assistant_id": "asst_JIkeRUrgk5urylj0H7jTil2w",
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "台北的天氣目前是22攝氏度。"
          },
          "type": "text"
        }
      ],
      "created_at": 1713953657,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_H0RJ1U2XOyHVP37wSKYSwjH8",
      "status": null,
      "thread_id": "thread_dlMEOzEUcNMRM49reFAR0eSN",
      "file_ids": []
    },
    {
      "id": "msg_TdWnjBeS6pEkMxoNpxDQq1GN",
      "assistant_id": null,
      "attachments": null,
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "台北天氣如何？"
          },
          "type": "text"
      